# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bredasdorp,-34.5322,20.0403,65.19,91,2,4.61,ZA,1707714216
1,1,codrington,-38.2667,141.9667,72.66,58,0,11.48,AU,1707714216
2,2,jamestown,42.0970,-79.2353,32.02,90,100,5.01,US,1707714053
3,3,yellowknife,62.4560,-114.3525,5.56,96,20,5.75,CA,1707714073
4,4,olonkinbyen,70.9221,-8.7187,28.00,64,100,10.07,SJ,1707714216


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [13]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City"
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [17]:
# Narrow down cities that fit criteria and drop any results with null values
#ideal_cities = (city_data_df["Max Temp"] >= 65) & (city_data_df["Max Temp"] <= 85)

#ideal_cities = city_data_df.query('Max Temp > 65 and Humidity < 70')
ideal_cities = city_data_df.loc[(city_data_df["Max Temp"] > 60) 
                                & (city_data_df["Max Temp"] < 80) 
                                & (city_data_df["Humidity"] < 70)
                                & (city_data_df["Humidity"] > 40)
                                & (city_data_df["Cloudiness"] < 50)
                                & (city_data_df["Wind Speed"] < 15)]
# Drop any rows with null values
ideal_cities.dropna(inplace = True)

# Display sample data
ideal_cities

C:\Users\nemuk\AppData\Local\Temp\ipykernel_25320\2999448166.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ideal_cities.dropna(inplace = True)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,codrington,-38.2667,141.9667,72.66,58,0,11.48,AU,1707714216
99,99,kerikeri,-35.2268,173.9474,74.91,65,18,1.01,NZ,1707714236
123,123,cabo san lucas,22.8909,-109.9124,66.76,62,2,1.45,MX,1707714242
166,166,sawakin,19.1059,37.3321,67.06,60,22,9.40,SD,1707714263
176,176,san patricio,28.0170,-97.5169,60.84,68,0,9.13,US,1707714266
198,198,hadibu,12.6500,54.0333,78.26,69,19,6.29,YE,1707714272
231,231,whangarei,-35.7317,174.3239,71.85,54,36,5.91,NZ,1707714280
287,287,gisborne,-38.6533,178.0042,63.37,59,5,8.63,NZ,1707714094
299,299,guerrero negro,27.9769,-114.0611,61.16,55,0,7.87,MX,1707714056
301,301,bubaque,11.2833,-15.8333,78.85,64,0,9.57,GW,1707714296


### Step 3: Create a new DataFrame called `hotel_df`.

In [18]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
1,codrington,AU,-38.2667,141.9667,58,
99,kerikeri,NZ,-35.2268,173.9474,65,
123,cabo san lucas,MX,22.8909,-109.9124,62,
166,sawakin,SD,19.1059,37.3321,60,
176,san patricio,US,28.0170,-97.5169,68,
198,hadibu,YE,12.6500,54.0333,69,
231,whangarei,NZ,-35.7317,174.3239,54,
287,gisborne,NZ,-38.6533,178.0042,59,
299,guerrero negro,MX,27.9769,-114.0611,55,
301,bubaque,GW,11.2833,-15.8333,64,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [20]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
codrington - nearest hotel: No hotel found
kerikeri - nearest hotel: No hotel found
cabo san lucas - nearest hotel: No hotel found
sawakin - nearest hotel: No hotel found
san patricio - nearest hotel: No hotel found
hadibu - nearest hotel: No hotel found
whangarei - nearest hotel: No hotel found
gisborne - nearest hotel: No hotel found
guerrero negro - nearest hotel: No hotel found
bubaque - nearest hotel: No hotel found
massawa - nearest hotel: No hotel found
howrah - nearest hotel: No hotel found
north shore - nearest hotel: No hotel found
el bauga - nearest hotel: No hotel found
zouerate - nearest hotel: No hotel found
hwange - nearest hotel: No hotel found
willowmore - nearest hotel: No hotel found
gwadar - nearest hotel: No hotel found
ishigaki - nearest hotel: No hotel found
naze - nearest hotel: No hotel found
machali - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
1,codrington,AU,-38.2667,141.9667,58,No hotel found
99,kerikeri,NZ,-35.2268,173.9474,65,No hotel found
123,cabo san lucas,MX,22.8909,-109.9124,62,No hotel found
166,sawakin,SD,19.1059,37.3321,60,No hotel found
176,san patricio,US,28.0170,-97.5169,68,No hotel found
198,hadibu,YE,12.6500,54.0333,69,No hotel found
231,whangarei,NZ,-35.7317,174.3239,54,No hotel found
287,gisborne,NZ,-38.6533,178.0042,59,No hotel found
299,guerrero negro,MX,27.9769,-114.0611,55,No hotel found
301,bubaque,GW,11.2833,-15.8333,64,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [37]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)